# Tool Call Accuracy Evaluator

## Objective
This sample demonstrates to how to use tool call accuracy evaluator on agent data. The supported input formats include:
- simple data such as strings;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 15 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name


The Tool Call Accuracy evaluator measures how accurately the agent called the correct tools to help resolve the user's query.
The scoring is on a 1-5 integer scale and is as follows:
 - Score 1: The tool calls are irrelevant
 - Score 2: The tool calls are partially relevant, but not enough tools were called or the parameters were not correctly passed.
 - Score 3: The tool calls are relevant, but there were unnecessary, excessive tool calls made.
 - Score 4: The tool calls are relevant, but some tools returned errors and agent retried calling them again and succeeded.
 - Score 5: The tool calls are relevant, and all parameters were correctly passed.


The evaluation requires the following inputs:

  - Query: The user query. Either a string with a user request or a list of messages with previous requests from the user and responses from the assistant, potentially including a system message.
  - ToolDefinitions: The list of tool definitions the agent can call. This is important for the evaluator to assess the accuracy of the tools calls made and the parameters passed.
  - And either one of:
    - Response: The response to be evaluated. Either a string or a message with the response from the agent to the last user query. If the response doesn't include tool calls made, then the evaluator will look into this parameter:
    - ToolCalls: The list of tool calls made by the agent for this run.

The result score is not_applicable if:
- No tool calls have been made in the run.
- Any or all of the tool calls that have been made do not have corresponding tool definitions.

### Initialize Tool Call Accuracy Evaluator


In [2]:
import os
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.ai.evaluation import ToolCallAccuracyEvaluator
from pprint import pprint

from dotenv import load_dotenv

# Load environment variables from .env file. Please provide your .env file
load_dotenv(override=True)

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

tool_call_accuracy_evaluator = ToolCallAccuracyEvaluator(model_config)

Class ToolCallAccuracyEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Samples

In [6]:
tool_definitions = [{'name': 'azure_maps_weather_current_conditions', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'lat': {'type': 'string', 'description': 'No description'}, 'lon': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_weather_daily_forecast', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'lat': {'type': 'string', 'description': 'No description'}, 'lon': {'type': 'string', 'description': 'No description'}, 'duration': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_weather_hourly_forecast', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'lat': {'type': 'string', 'description': 'No description'}, 'lon': {'type': 'string', 'description': 'No description'}, 'duration': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_haversine_distance', 'description': 'Calculate the Haversine distance between two points on the Earth specified in decimal degrees.', 'parameters': {'type': 'object', 'properties': {'lat1': {'type': 'number', 'description': 'No description'}, 'lon1': {'type': 'number', 'description': 'No description'}, 'lat2': {'type': 'number', 'description': 'No description'}, 'lon2': {'type': 'number', 'description': 'No description'}, 'unit': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_search', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'address': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_fuzzy_search', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_reverse_address_search', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'lat': {'type': 'string', 'description': 'No description'}, 'lon': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_point_of_interest_search', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_geolocation', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'ip_address': {'type': 'string', 'description': 'No description'}}}}, {'name': 'azure_maps_timezone', 'description': 'No description', 'parameters': {'type': 'object', 'properties': {'lat': {'type': 'number', 'description': 'No description'}, 'lon': {'type': 'number', 'description': 'No description'}}}}]

In [ ]:
# no tool calls found in the response
evalfunc = tool_call_accuracy_evaluator

result = evalfunc(
    query="What are the opening hours of the Eiffel Tower?",
    response="Opening hours of the Eiffel Tower are 9:00 AM to 11:00 PM.",
    tool_definitions=[],
)

pprint(result, sort_dicts=False)

In [ ]:
# no tool calls in the tool_calls argument
result = evalfunc(
    query="What are the opening hours of the Eiffel Tower?",
    tool_calls=[],
    tool_definitions=[],
)

pprint(result, sort_dicts=False)

In [ ]:
# no tool definitions provided
result = evalfunc(
    query='What is the timezone for the coordinates 41.8781, -87.6298?',
    tool_calls=[
            {
                "type": "tool_call",
                "name": "azure_maps_timezone",
                "arguments": {'lat': 41.8781, 'lon': -87.6298},
            },
        ],
    tool_definitions=[],
)

pprint(result, sort_dicts=False)

In [ ]:
# Success example. Tool calls were correct so were the parameters passed to the tools.
result = evalfunc(
    query='What is the timezone for the coordinates 41.8781, -87.6298?',
    tool_definitions=tool_definitions,
    response=[
            {'createdAt': '2025-04-26T05:03:33Z',
            'run_id': 'run_rqDJ1gNotLDqHBoDPBZbnjNf',
            'role': 'assistant',
            'content': [{'type': 'tool_call',
                        'tool_call_id': 'call_SwKcfU1T6NsxGgqs32OmqOZ3',
                        'name': 'azure_maps_timezone',
                        'arguments': {'lat': 41.8781, 'lon': -87.6298}}]},
            {'createdAt': '2025-04-26T05:03:35Z',
            'run_id': 'run_rqDJ1gNotLDqHBoDPBZbnjNf',
            'tool_call_id': 'call_SwKcfU1T6NsxGgqs32OmqOZ3',
            'role': 'tool',
            'content': [{'type': 'tool_result',
                        'tool_result': {'ianaId': 'America/Chicago',
                                        'utcOffset': None,
                                        'abbreviation': None,
                                        'isDaylightSavingTime': None}}]},
            {'createdAt': '2025-04-26T05:03:36Z',
            'run_id': 'run_rqDJ1gNotLDqHBoDPBZbnjNf',
            'role': 'assistant',
            'content': [{'type': 'text',
                        'text': 'The timezone for the coordinates 41.8781, -87.6298 is '
                                '**America/Chicago**.'}]}
        ]
)

pprint(result, sort_dicts=False)

In [ ]:
result = evalfunc(
    query="What is the distance between San Francisco, CA and Johannesburg, South Africa?",
    tool_calls=[
        {
            "type": "tool_call",
            "name": "azure_maps_fuzzy_search",
            "arguments": {
            "query": "San Francisco, CA"
            }
        },
        {
            "type": "tool_call",
            "name": "azure_maps_fuzzy_search",
            "arguments": {
            "query": "Johannesburg, South Africa"
            }
        },
        {
            "type": "tool_call",
            "name": "azure_maps_haversine_distance",
            "arguments": {
            "lat1": 37.780077,
            "lon1": -122.420162,
            "lat2": -26.201452,
            "lon2": 28.045488
            }
        }
    ],
    tool_definitions=tool_definitions,
)
pprint(result, sort_dicts=False)

In [ ]:
# Fail example.
result = evalfunc(
    query="What if the weather in Georgetown, TX for the next 3 days?",
    response=[
        {'createdAt': '2025-04-26T06:09:59Z',
        'run_id': 'run_4MbNisDrdaIUhErbz3kp1HrU',
        'role': 'assistant',
        'content': [{'type': 'tool_call',
                    'tool_call_id': 'call_0dgzpQvXBbsmVZlcJWW7C5st',
                    'name': 'azure_maps_fuzzy_search',
                    'arguments': {'query': 'Georgetown'}}]},
        {'createdAt': '2025-04-26T06:10:02Z',
        'run_id': 'run_4MbNisDrdaIUhErbz3kp1HrU',
        'tool_call_id': 'call_0dgzpQvXBbsmVZlcJWW7C5st',
        'role': 'tool',
        'content': [{'type': 'tool_result',
                    'tool_result': {'address': 'Georgetown, TX',
                                    'position': {'lat': 30.63325,
                                                'lon': -97.676978}}}]
        },
        {'run_id': 'run_4MbNisDrdaIUhErbz3kp1HrU',
        'role': 'assistant',
        'content': [{'type': 'tool_call',
                    'tool_call_id': 'call_xUVoGdu7LTcVe9IexgcA1HSt',
                    'name': 'azure_maps_weather_daily_forecast',
                    'arguments': {'lat': '30.63325',
                                    'lon': '-97.676978',
                                    'duration': '3'}}]
        },
        {'createdAt': '2025-04-26T03:23:56Z',
        'run_id': 'run_4MbNisDrdaIUhErbz3kp1HrU',
        'tool_call_id': 'call_xUVoGdu7LTcVe9IexgcA1HSt',
        'role': 'tool',
        'content': [{'type': 'tool_result',
                    'tool_result': {'error': 'Request failed with status code '
                                                '400'}}]},
        {'createdAt': '2025-04-26T03:23:57Z',
        'run_id': 'run_4MbNisDrdaIUhErbz3kp1HrU',
        'role': 'assistant',
        'content': [{'type': 'text',
                    'text': 'I encountered an error while trying to retrieve the '
                            '3-day weather forecast for Georgetown, TX. \n'
                            '\n'
                            'Would you like me to try again or provide any other '
                            'specific information?'}]}
        ],
    tool_definitions=tool_definitions)

pprint(result, sort_dicts=False)

In [ ]:
query = [
    {"role": "system", "content": "You are a friendly and helpful customer service agent."},
    {
        "createdAt": "2025-03-14T06:14:20Z",
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Hi, I need help with the last 2 orders on my account #888. Could you please update me on their status?",
            }
        ],
    },
]

response = [
    {
        "createdAt": "2025-03-14T06:14:30Z",
        "run_id": "0",
        "role": "assistant",
        "content": [{"type": "text", "text": "Hello! Let me quickly look up your account details."}],
    },
    {
        "createdAt": "2025-03-14T06:14:35Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_001",
                "name": "get_orders",
                "arguments": {"account_number": "888"},
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:40Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_001",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": '[{ "order_id": "123" }, { "order_id": "124" }]'}],
    },
    {
        "createdAt": "2025-03-14T06:14:45Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Thanks for your patience. I see two orders on your account. Let me fetch the details for both.",
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:50Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_002",
                "name": "get_order",
                "arguments": {"order_id": "123"},
            },
            {
                "type": "tool_call",
                "tool_call_id": "tool_call_20250310_003",
                "name": "get_order",
                "arguments": {"order_id": "124"},
            },
        ],
    },
    {
        "createdAt": "2025-03-14T06:14:55Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_002",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": '{ "order": { "id": "123", "status": "shipped", "delivery_date": "2025-03-15" } }',
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:15:00Z",
        "run_id": "0",
        "tool_call_id": "tool_call_20250310_003",
        "role": "tool",
        "content": [
            {
                "type": "tool_result",
                "tool_result": '{ "order": { "id": "124", "status": "delayed", "expected_delivery": "2025-03-20" } }',
            }
        ],
    },
    {
        "createdAt": "2025-03-14T06:15:05Z",
        "run_id": "0",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The order with ID 123 has been shipped and is expected to be delivered on March 15, 2025. However, the order with ID 124 is delayed and should now arrive by March 20, 2025. Is there anything else I can help you with?",
            }
        ],
    },
]

tool_definitions = [
    {
        "name": "get_orders",
        "description": "Get the list of orders for a given account number.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {"type": "string", "description": "The account number to get the orders for."}
            },
        },
    },
    {
        "name": "get_order",
        "description": "Get the details of a specific order.",
        "parameters": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID to get the details for."}},
        },
    },
    {
        "name": "initiate_return",
        "description": "Initiate the return process for an order.",
        "parameters": {
            "type": "object",
            "properties": {"order_id": {"type": "string", "description": "The order ID for the return process."}},
        },
    },
    {
        "name": "update_shipping_address",
        "description": "Update the shipping address for a given account.",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {"type": "string", "description": "The account number to update."},
                "new_address": {"type": "string", "description": "The new shipping address."},
            },
        },
    },
]

result = evalfunc(
    query=query,
    response=response,
    tool_definitions=tool_definitions,
)
pprint(result, sort_dicts=False)